# MATH 468 - Stochastic Calculus II - HW 4

In [1]:
'''Packages Used'''
import pandas as pd
import numpy as np

## Problem 1 (American options with N large)

*Note*: This problem is a continuation of Problem 3 in HW #3. The objective is to generalize your code to American options. Please provide the answers to the questions below as well as a sample of your code.

We consider the same market model as in Problem 3 of HW #3. We consider an American put option with strike price $K = \$1,025$ and maturity $T = 1$ year.

(a) Find the price of the option at time $t_0 = 0$ if it is American and compare with the price of the European put found in Problem 1.  
(b) What is the earliest possible time of optimal exercise of the American put?  
(c) Repeat questions (a) and (b) in the case where $r = 0\%$. What do you conclude?  

### Solution to Part (a)

In [2]:
'''Code Block'''

'Code Block'

### Solution to Part (b)

In [3]:
'''Code Block'''

'Code Block'

### Solution to Part (c)

In [4]:
'''Code Block'''

'Code Block'

## Problem 2 (Option pricing from real data.)

*Note*: This problem is the continuation of Prob. 3 in HW #1. We will use the data to price options and compare with the market prices.

Consider the asset that you selected in HW #1. For this asset, select three call options and one put option. All options should have approximately the same expiration date, say about 3 months in the future. The three calls should have three different strike prices, typically one *in-the-money*, one *at-the-money*, and one *out-of-the-money*. (Consider values relatively close to the current price and, if you have access to this information, for which the trading volume is relatively large.) The put option should be *at-the-money*. Record the current price of these options. Note that all traded options are American options. Also determine the interest rate to be used for your calculations: typically, you can consider the LIBOR rate, or the 10-year US treasury yield.

(a) We assume that the CRR model represents the stock price and that its assumptions are satisfied. Based on historical data (about 3 months to a year in the past), determine the parameters $\mu$ and $\sigma$ for the stock that you chose following the methods discussed in class. (I recommend to download current data, but feel free to use past data if that is all you have.)  
(b) Price the selected put option using an $N$-period binomial model for different values of $N$ (which you can choose on your own). As a hint, you can for instance start with $N = 2$ or $N = 3$, which should be approximately monthly-periods. You can then use approximately weekly periods ($N \approx 10$ or $N \approx 12$) or daily periods ($N \approx 60$). Make sure that you use the exact number of days or weeks to the maturity of your option.  
(c) Compare your results to the market price and comment. In particular, discuss the impact of $N$ on your calculated price.  
(d) Price each of the three selected call options using all of (i) approximately monthly periods ($N = 2, 3$), (ii) approximately weekly periods ($N \approx 10, 12$), and (iii) daily periods ($N \approx 60$).  
(e) Compare your results to the market prices and comment. In particular, discuss the impact of the strike price $K$ on your results. In general, once you obtain the parameters $\mu$ and $\sigma$, you should be able to use the computing tool that you developed in HW#3 and in Problem 1 to price your options.


### Solution to Part (a)

In [5]:
'''Code Block'''

'Code Block'

### Solution to Part (b)

In [6]:
'''Code Block'''

'Code Block'

### Solution to Part (c)

In [7]:
'''Code Block'''

'Code Block'

### Solution to Part (d)

In [8]:
'''Code Block'''

'Code Block'

### Solution to Part (e)

In [9]:
'''Code Block'''

'Code Block'

## Problem 3

We consider an N-period binomial model with the following properties: each period lasts a year and the current stock price is S0 = 4. On each period, the stock price doubles when it moves up and is reduced by half when it moves down. The annual interest rate on the money market is $25\%$.

(a) Find the values of $\mu$ and $\sigma$ associated with this CRR model.  
(b) Assuming that $p = \frac{1}{2}$, find $P(S_T > 6)$ when $N = 3$ and $N = 10$.  
(c) Using the risk-neutral option pricing formula (written as an expectation), calculate the price of a European call with strike price $K = 8$ when $N = 3$ and $N = 10$.


### Solution to Part (a)

In [10]:
'''Code Block'''

'Code Block'

### Solution to Part (b)

In [11]:
'''Code Block'''

'Code Block'

### Solution to Part (c)

In [12]:
'''Code Block'''

'Code Block'